In [8]:
import torch
print(torch.cuda.is_available())  # For PyTorch
import os
import shutil
import random

# Define source and destination paths
source_folder = r"D:\git\FYP\Python\TwoStage\data\West_Indian"
dest_folder = r"D:\git\FYP\Python\TwoStage\data\West_Indian_7030SPLIT"

# Define train and val directories
train_folder = os.path.join(dest_folder, "train")
val_folder = os.path.join(dest_folder, "val")

# Create train and val directories if they don’t exist
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)

# Get all images in the source folder
images = [f for f in os.listdir(source_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Shuffle images randomly
random.shuffle(images)

# Split into 70% train, 30% val
split_index = int(0.7 * len(images))
train_images = images[:split_index]
val_images = images[split_index:]

# Move images to respective folders
for img in train_images:
    shutil.copy(os.path.join(source_folder, img), os.path.join(train_folder, img))

for img in val_images:
    shutil.copy(os.path.join(source_folder, img), os.path.join(val_folder, img))

print(f"Copied {len(train_images)} images to train and {len(val_images)} images to val.")


True
Copied 145 images to train and 63 images to val.


In [10]:
import os
import cv2
from tqdm import tqdm

def preprocess_and_save_patches(input_dir, output_dir, patch_size=(224, 224)):
    """
    Processes images into 5 patches and saves them in a structured folder format.
    
    Output folder structure:
    output_dir/
        image_1/
            image_1_patch_0.jpg
            image_1_patch_1.jpg
            image_1_patch_2.jpg
            image_1_patch_3.jpg
            image_1_patch_4.jpg
        image_2/
            ...
    """
    os.makedirs(output_dir, exist_ok=True)

    for img_file in tqdm(os.listdir(input_dir), desc="Processing images"):
        img_path = os.path.join(input_dir, img_file)
        image = cv2.imread(img_path)

        if image is None:
            print(f"Skipping {img_path}: Unable to read")
            continue

        # Create folder for each image
        image_name = os.path.splitext(img_file)[0]
        image_output_path = os.path.join(output_dir, image_name)
        os.makedirs(image_output_path, exist_ok=True)

        # Split image into 5 patches
        height, width = image.shape[:2]
        mid_x, mid_y = width // 2, height // 2
        start_x, end_x = int(0.25 * width), int(0.75 * width)
        start_y, end_y = int(0.25 * height), int(0.75 * height)

        patches = [
            image[0:mid_y, 0:mid_x],  # Top-left
            image[0:mid_y, mid_x:width],  # Top-right
            image[mid_y:height, 0:mid_x],  # Bottom-left
            image[mid_y:height, mid_x:width],  # Bottom-right
            image[start_y:end_y, start_x:end_x],  # Center
        ]

        # Resize and save each patch
        for i, patch in enumerate(patches):
            resized_patch = cv2.resize(patch, patch_size)
            patch_filename = f"{image_name}_patch_{i}.jpg"
            patch_path = os.path.join(image_output_path, patch_filename)
            cv2.imwrite(patch_path, resized_patch)

    print("✅ Patching complete! Check:", output_dir)

# Define input and output directories
input_directory = r"D:\git\FYP\Python\TwoStage\data\West_Indian"
output_directory = r"D:\git\FYP\Python\TwoStage\data\West_Indian_Patches"

# Run the function
preprocess_and_save_patches(input_directory, output_directory)


Processing images: 100%|██████████| 208/208 [00:07<00:00, 27.18it/s]

✅ Patching complete! Check: D:\git\FYP\Python\TwoStage\data\West_Indian_Patches


to split the images into patches

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
from PIL import Image
import cv2

In [ ]:
#for testing if teh sample are being segmnted correctly
import os
import cv2
from tqdm import tqdm

def preprocess_and_save_patches(input_dir, output_dir, patch_size=(224, 224)):
    """
    Preprocesses images into patches and saves them in a structured folder format.
    
    Folder structure:
    output_dir/
        train/
            class_1/
                image_1/  # Folder containing all patches of image_1
                    image_1_patch_0.jpg
                    image_1_patch_1.jpg
                    ...
                    image_1_resized.jpg  # The resized original image
                image_2/
                    ...
            class_2/
                ...
        val/
            ...
        test/
            ...
    """
    os.makedirs(output_dir, exist_ok=True)

    for split in ["West_Indian"]:  # Loop through dataset splits
        split_input_dir = os.path.join(input_dir, split)
        split_output_dir = os.path.join(output_dir, split)
        os.makedirs(split_output_dir, exist_ok=True)

        for label_dir in os.listdir(split_input_dir):
            label_path = os.path.join(split_input_dir, label_dir)
            if not os.path.isdir(label_path):
                continue

            # Create directory for label in output
            output_label_path = os.path.join(split_output_dir, label_dir)
            os.makedirs(output_label_path, exist_ok=True)

            for img_file in tqdm(os.listdir(label_path), desc=f"Processing {split}/{label_dir}"):
                img_path = os.path.join(label_path, img_file)
                image = cv2.imread(img_path)
                
                if image is None:
                    print(f"Skipping {img_path}: Unable to read")
                    continue

                # Create subfolder for each image's patches
                image_name = os.path.splitext(img_file)[0]
                image_output_path = os.path.join(output_label_path, image_name)
                os.makedirs(image_output_path, exist_ok=True)

                # Split into patches
                height, width = image.shape[:2]
                mid_x, mid_y = width // 2, height // 2
                start_x, end_x = int(0.25 * width), int(0.75 * width)
                start_y, end_y = int(0.25 * height), int(0.75 * height)

                patches = [
                    image[0:mid_y, 0:mid_x],  # Top-left
                    image[0:mid_y, mid_x:width],  # Top-right
                    image[mid_y:height, 0:mid_x],  # Bottom-left
                    image[mid_y:height, mid_x:width],  # Bottom-right
                    image[start_y:end_y, start_x:end_x],  # Center
                ]

                # Resize and save each patch
                for i, patch in enumerate(patches):
                    resized_patch = cv2.resize(patch, patch_size)
                    patch_filename = f"{image_name}_patch_{i}.jpg"
                    patch_path = os.path.join(image_output_path, patch_filename)
                    cv2.imwrite(patch_path, resized_patch)

                # Resize and save the original image as well
                # resized_original_image = cv2.resize(image, patch_size)
                # original_filename = f"{image_name}_resized.jpg"
                # original_image_path = os.path.join(image_output_path, original_filename)
                # cv2.imwrite(original_image_path, resized_original_image)

# Define input and output directories
input_directory = "D:\git\FYP\Python\TwoStage\data"
output_directory = "D:\git\FYP\Python\TwoStage\data\West_Indian_Patches"



# Run the preprocessing function
preprocess_and_save_patches(input_directory, output_directory)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:\\git\\FYP\\Python\\TwoStage\\data\\all_images'

In [7]:
#for testing if teh sample are being segmnted correctly 80 split
import os
import cv2
from tqdm import tqdm

def preprocess_and_save_patches(input_dir, output_dir, patch_size=(224, 224)):
    """
    Preprocesses images into patches and saves them in a structured folder format.
    
    Folder structure:
    output_dir/
        train/
            class_1/
                image_1/  # Folder containing all patches of image_1
                    image_1_patch_0.jpg
                    image_1_patch_1.jpg
                    ...
                    image_1_resized.jpg  # The resized original image
                image_2/
                    ...
            class_2/
                ...
        val/
            ...
        test/
            ...
    """
    os.makedirs(output_dir, exist_ok=True)

    for split in ["train", "val", "test"]:  # Loop through dataset splits
        split_input_dir = os.path.join(input_dir, split)
        split_output_dir = os.path.join(output_dir, split)
        os.makedirs(split_output_dir, exist_ok=True)

        for label_dir in os.listdir(split_input_dir):
            label_path = os.path.join(split_input_dir, label_dir)
            if not os.path.isdir(label_path):
                continue

            # Create directory for label in output
            output_label_path = os.path.join(split_output_dir, label_dir)
            os.makedirs(output_label_path, exist_ok=True)

            for img_file in tqdm(os.listdir(label_path), desc=f"Processing {split}/{label_dir}"):
                img_path = os.path.join(label_path, img_file)
                image = cv2.imread(img_path)
                
                if image is None:
                    print(f"Skipping {img_path}: Unable to read")
                    continue

                # Create subfolder for each image's patches
                image_name = os.path.splitext(img_file)[0]
                image_output_path = os.path.join(output_label_path, image_name)
                os.makedirs(image_output_path, exist_ok=True)

                # Split into patches
                height, width = image.shape[:2]
                mid_x, mid_y = width // 2, height // 2
                start_x, end_x = int(0.25 * width), int(0.75 * width)
                start_y, end_y = int(0.25 * height), int(0.75 * height)

                patches = [
                    image[0:mid_y, 0:mid_x],  # Top-left
                    image[0:mid_y, mid_x:width],  # Top-right
                    image[mid_y:height, 0:mid_x],  # Bottom-left
                    image[mid_y:height, mid_x:width],  # Bottom-right
                    image[start_y:end_y, start_x:end_x],  # Center
                ]

                # Resize and save each patch
                for i, patch in enumerate(patches):
                    resized_patch = cv2.resize(patch, patch_size)
                    patch_filename = f"{image_name}_patch_{i}.jpg"
                    patch_path = os.path.join(image_output_path, patch_filename)
                    cv2.imwrite(patch_path, resized_patch)

                # Resize and save the original image as well
                # resized_original_image = cv2.resize(image, patch_size)
                # original_filename = f"{image_name}_resized.jpg"
                # original_image_path = os.path.join(image_output_path, original_filename)
                # cv2.imwrite(original_image_path, resized_original_image)

# Define input and output directories
input_directory = "D:\git\FYP\Python\TwoStage\data\Pandora_7k\Pandora7k_split_80"
output_directory = "D:\git\FYP\Python\TwoStage\data\Pandora_7k\Pandora7k_split_80_patches"

# Run the preprocessing function
preprocess_and_save_patches(input_directory, output_directory)


Processing test/Surrealism: 100%|██████████| 25/25 [00:02<00:00,  9.53it/s]


In [8]:
#for testing if teh sample are being segmnted correctly 70 split
import os
import cv2
from tqdm import tqdm

def preprocess_and_save_patches(input_dir, output_dir, patch_size=(224, 224)):
    """
    Preprocesses images into patches and saves them in a structured folder format.
    
    Folder structure:
    output_dir/
        train/
            class_1/
                image_1/  # Folder containing all patches of image_1
                    image_1_patch_0.jpg
                    image_1_patch_1.jpg
                    ...
                    image_1_resized.jpg  # The resized original image
                image_2/
                    ...
            class_2/
                ...
        val/
            ...
        test/
            ...
    """
    os.makedirs(output_dir, exist_ok=True)

    for split in ["train", "val", "test"]:  # Loop through dataset splits
        split_input_dir = os.path.join(input_dir, split)
        split_output_dir = os.path.join(output_dir, split)
        os.makedirs(split_output_dir, exist_ok=True)

        for label_dir in os.listdir(split_input_dir):
            label_path = os.path.join(split_input_dir, label_dir)
            if not os.path.isdir(label_path):
                continue

            # Create directory for label in output
            output_label_path = os.path.join(split_output_dir, label_dir)
            os.makedirs(output_label_path, exist_ok=True)

            for img_file in tqdm(os.listdir(label_path), desc=f"Processing {split}/{label_dir}"):
                img_path = os.path.join(label_path, img_file)
                image = cv2.imread(img_path)
                
                if image is None:
                    print(f"Skipping {img_path}: Unable to read")
                    continue

                # Create subfolder for each image's patches
                image_name = os.path.splitext(img_file)[0]
                image_output_path = os.path.join(output_label_path, image_name)
                os.makedirs(image_output_path, exist_ok=True)

                # Split into patches
                height, width = image.shape[:2]
                mid_x, mid_y = width // 2, height // 2
                start_x, end_x = int(0.25 * width), int(0.75 * width)
                start_y, end_y = int(0.25 * height), int(0.75 * height)

                patches = [
                    image[0:mid_y, 0:mid_x],  # Top-left
                    image[0:mid_y, mid_x:width],  # Top-right
                    image[mid_y:height, 0:mid_x],  # Bottom-left
                    image[mid_y:height, mid_x:width],  # Bottom-right
                    image[start_y:end_y, start_x:end_x],  # Center
                ]

                # Resize and save each patch
                for i, patch in enumerate(patches):
                    resized_patch = cv2.resize(patch, patch_size)
                    patch_filename = f"{image_name}_patch_{i}.jpg"
                    patch_path = os.path.join(image_output_path, patch_filename)
                    cv2.imwrite(patch_path, resized_patch)

                # Resize and save the original image as well
                # resized_original_image = cv2.resize(image, patch_size)
                # original_filename = f"{image_name}_resized.jpg"
                # original_image_path = os.path.join(image_output_path, original_filename)
                # cv2.imwrite(original_image_path, resized_original_image)

# Define input and output directories
input_directory = "D:\git\FYP\Python\TwoStage\data\Pandora_7k\Pandora7k_split_70"
output_directory = "D:\git\FYP\Python\TwoStage\data\Pandora_7k\Pandora7k_split_70_patches"

# Run the preprocessing function
preprocess_and_save_patches(input_directory, output_directory)


Processing test/Surrealism: 100%|██████████| 37/37 [00:00<00:00, 46.99it/s]


In [9]:
import os
import shutil

def rename_folders(directory):
    # List all items in the directory
    for root, dirs, files in os.walk(directory):
        for dir_name in dirs:
            # Check if the folder name starts with 2 digits and underscore
            if len(dir_name) >= 3 and dir_name[0:2].isdigit() and dir_name[2] == '_':
                # Create new name by removing first 3 characters
                new_name = dir_name[3:]
                
                # Create full paths
                old_path = os.path.join(root, dir_name)
                new_path = os.path.join(root, new_name)
                
                # Rename the folder
                try:
                    os.rename(old_path, new_path)
                    print(f"Renamed: {dir_name} -> {new_name}")
                except Exception as e:
                    print(f"Error renaming {dir_name}: {e}")

# Specify the directory to process
directory_to_process = r'D:\git\FYP\Python\TwoStage\data\dataset3\images'
rename_folders(directory_to_process)


Renamed: 01_Byzantin_Iconography -> Byzantin_Iconography
Renamed: 02_Early_Renaissance -> Early_Renaissance
Renamed: 03_Northern_Renaissance -> Northern_Renaissance
Renamed: 04_High_Renaissance -> High_Renaissance
Renamed: 05_Baroque -> Baroque
Renamed: 06_Rococo -> Rococo
Renamed: 07_Romanticism -> Romanticism
Renamed: 08_Realism -> Realism
Renamed: 09_Impressionism -> Impressionism
Renamed: 10_Post_Impressionism -> Post_Impressionism
Renamed: 11_Expressionism -> Expressionism
Renamed: 12_Symbolism -> Symbolism
Renamed: 13_Fauvism -> Fauvism
Renamed: 14_Cubism -> Cubism
Renamed: 15_Surrealism -> Surrealism
Renamed: 16_AbstractArt -> AbstractArt
Renamed: 17_NaiveArt -> NaiveArt
Renamed: 18_PopArt -> PopArt
